# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 16 2023 1:24PM,259459,10,DNMD-36735,Emerginnova,Released
1,Mar 16 2023 1:21PM,259457,10,HH-40657,Hush Hush,Released
2,Mar 16 2023 1:21PM,259457,10,HH-40658,Hush Hush,Released
3,Mar 16 2023 1:21PM,259457,10,HH-40659,Hush Hush,Released
4,Mar 16 2023 1:18PM,259458,10,MSP219608,"Methapharm, Inc.",Released
5,Mar 16 2023 1:18PM,259458,10,MSP219609,"Methapharm, Inc.",Released
6,Mar 16 2023 1:18PM,259458,10,MSP219612,"Methapharm, Inc.",Released
7,Mar 16 2023 1:18PM,259458,10,MSP219613,"Methapharm, Inc.",Released
8,Mar 16 2023 1:18PM,259458,10,MSP219614,"Methapharm, Inc.",Released
9,Mar 16 2023 1:18PM,259458,10,MSP219615,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,259452,Released,2
15,259455,Released,1
16,259457,Released,3
17,259458,Released,7
18,259459,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
259452,NaN,14.0,2.0
259455,NaN,NaN,1.0
259457,NaN,NaN,3.0
259458,NaN,NaN,7.0
259459,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259385,0.0,6.0,0.0
259414,3.0,46.0,5.0
259416,0.0,0.0,6.0
259433,0.0,2.0,3.0
259436,0.0,1.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259385,0,6,0
259414,3,46,5
259416,0,0,6
259433,0,2,3
259436,0,1,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259385,0,6,0
1,259414,3,46,5
2,259416,0,0,6
3,259433,0,2,3
4,259436,0,1,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259385,,6,
1,259414,3,46,5
2,259416,,,6
3,259433,,2,3
4,259436,,1,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 16 2023 1:24PM,259459,10,Emerginnova
1,Mar 16 2023 1:21PM,259457,10,Hush Hush
4,Mar 16 2023 1:18PM,259458,10,"Methapharm, Inc."
11,Mar 16 2023 12:59PM,259455,16,Sartorius Bioprocess Solutions
12,Mar 16 2023 12:05PM,259452,10,Bio-PRF
28,Mar 16 2023 11:56AM,259451,19,"NAPP Technologies, LLC"
29,Mar 16 2023 11:49AM,259450,19,"GCH Granules USA, Inc."
35,Mar 16 2023 10:35AM,259439,19,"GCH Granules USA, Inc."
41,Mar 16 2023 10:22AM,259436,19,"GCH Granules USA, Inc."
46,Mar 16 2023 10:18AM,259385,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 16 2023 1:24PM,259459,10,Emerginnova,,,1
1,Mar 16 2023 1:21PM,259457,10,Hush Hush,,,3
2,Mar 16 2023 1:18PM,259458,10,"Methapharm, Inc.",,,7
3,Mar 16 2023 12:59PM,259455,16,Sartorius Bioprocess Solutions,,,1
4,Mar 16 2023 12:05PM,259452,10,Bio-PRF,,14,2
5,Mar 16 2023 11:56AM,259451,19,"NAPP Technologies, LLC",,,1
6,Mar 16 2023 11:49AM,259450,19,"GCH Granules USA, Inc.",,,6
7,Mar 16 2023 10:35AM,259439,19,"GCH Granules USA, Inc.",,1,5
8,Mar 16 2023 10:22AM,259436,19,"GCH Granules USA, Inc.",,1,4
9,Mar 16 2023 10:18AM,259385,15,"Alliance Pharma, Inc.",,6,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 16 2023 1:24PM,259459,10,Emerginnova,1,,
1,Mar 16 2023 1:21PM,259457,10,Hush Hush,3,,
2,Mar 16 2023 1:18PM,259458,10,"Methapharm, Inc.",7,,
3,Mar 16 2023 12:59PM,259455,16,Sartorius Bioprocess Solutions,1,,
4,Mar 16 2023 12:05PM,259452,10,Bio-PRF,2,14,
5,Mar 16 2023 11:56AM,259451,19,"NAPP Technologies, LLC",1,,
6,Mar 16 2023 11:49AM,259450,19,"GCH Granules USA, Inc.",6,,
7,Mar 16 2023 10:35AM,259439,19,"GCH Granules USA, Inc.",5,1,
8,Mar 16 2023 10:22AM,259436,19,"GCH Granules USA, Inc.",4,1,
9,Mar 16 2023 10:18AM,259385,15,"Alliance Pharma, Inc.",,6,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 16 2023 1:24PM,259459,10,Emerginnova,1,,
1,Mar 16 2023 1:21PM,259457,10,Hush Hush,3,,
2,Mar 16 2023 1:18PM,259458,10,"Methapharm, Inc.",7,,
3,Mar 16 2023 12:59PM,259455,16,Sartorius Bioprocess Solutions,1,,
4,Mar 16 2023 12:05PM,259452,10,Bio-PRF,2,14,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 16 2023 1:24PM,259459,10,Emerginnova,1.0,NaN,NaN
1,Mar 16 2023 1:21PM,259457,10,Hush Hush,3.0,NaN,NaN
2,Mar 16 2023 1:18PM,259458,10,"Methapharm, Inc.",7.0,NaN,NaN
3,Mar 16 2023 12:59PM,259455,16,Sartorius Bioprocess Solutions,1.0,NaN,NaN
4,Mar 16 2023 12:05PM,259452,10,Bio-PRF,2.0,14.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 16 2023 1:24PM,259459,10,Emerginnova,1.0,0.0,0.0
1,Mar 16 2023 1:21PM,259457,10,Hush Hush,3.0,0.0,0.0
2,Mar 16 2023 1:18PM,259458,10,"Methapharm, Inc.",7.0,0.0,0.0
3,Mar 16 2023 12:59PM,259455,16,Sartorius Bioprocess Solutions,1.0,0.0,0.0
4,Mar 16 2023 12:05PM,259452,10,Bio-PRF,2.0,14.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1037826,13.0,14.0,0.0
15,259385,0.0,6.0,0.0
16,259455,1.0,0.0,0.0
19,1816039,30.0,50.0,3.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1037826,13.0,14.0,0.0
1,15,259385,0.0,6.0,0.0
2,16,259455,1.0,0.0,0.0
3,19,1816039,30.0,50.0,3.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,13.0,14.0,0.0
1,15,0.0,6.0,0.0
2,16,1.0,0.0,0.0
3,19,30.0,50.0,3.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,13.0
1,15,Released,0.0
2,16,Released,1.0
3,19,Released,30.0
4,10,Executing,14.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19
Status,,,,
Completed,0.0,0.0,0.0,3.0
Executing,14.0,6.0,0.0,50.0
Released,13.0,0.0,1.0,30.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19
0,Completed,0.0,0.0,0.0,3.0
1,Executing,14.0,6.0,0.0,50.0
2,Released,13.0,0.0,1.0,30.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19
0,Completed,0.0,0.0,0.0,3.0
1,Executing,14.0,6.0,0.0,50.0
2,Released,13.0,0.0,1.0,30.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()